In this Notebook we fetch Reddit posts and comments using Reddit API

In [ ]:
# installing required libraries
!pip install praw


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# Define functions to scrape data from Reddit API
import praw
import pandas as pd

def setup_reddit_api(client_id, client_secret, user_agent):
    """Set up Reddit API client."""
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent
    )
    return reddit

def get_subreddit_posts(reddit, subreddit_name, limit=100):
    """Get posts from a subreddit."""
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    for post in subreddit.hot(limit=limit):
        posts.append({
            'post_id': post.id,
            'title': post.title,
            'text': post.selftext,
            'author': str(post.author),
            'created_utc': post.created_utc,
            'score': post.score,
            'num_comments': post.num_comments,
            'upvote_ratio': post.upvote_ratio
        })

    return pd.DataFrame(posts)

def get_post_comments(reddit, post_id, limit=100):
    """Get comments for a specific post."""
    post = reddit.submission(id=post_id)
    post.comments.replace_more(limit=0)  # Only get top-level comments
    comments = []

    for comment in post.comments[:limit]:
        comments.append({
            'comment_id': comment.id,
            'post_id': post_id,
            'author': str(comment.author),
            'text': comment.body,
            'score': comment.score,
            'created_utc': comment.created_utc
        })

    return pd.DataFrame(comments)

In [18]:
# Connect to Reddit API
client_id = "YI3yiSbD9yctHZ2NMIQoZA"
client_secret = "vZ1MFPXuDEXCe-NaEwEyKnDrKyaPyg"
user_agent = "script : data_collection :v1 .0 (by/u/data_collection)"
reddit = setup_reddit_api( client_id , client_secret , user_agent )


In [20]:
# Scrape Reddit posts
import os

posts_df_reddit = get_subreddit_posts(reddit , "datascience" , limit =50)

output_dir = '../reddit_data'
os.makedirs(output_dir, exist_ok=True)

# save posts_df_reddit to csv
posts_df_reddit.to_csv(f'{output_dir}/reddit_posts_df.csv', index=False)

# view posts_df_reddit
posts_df_reddit.head()

,post_id,title,text,author,created_utc,score,num_comments,upvote_ratio
0,1k44mgg,Weekly Entering & Transitioning - Thread 21 Ap...,\n\nWelcome to this week's entering & transit...,AutoModerator,1.745208e+09,4,2,1.00
1,1i5inrb,Weekly Entering & Transitioning - Thread 20 Ja...,\n\nWelcome to this week's entering & transit...,AutoModerator,1.737349e+09,14,46,1.00
2,1k4geso,Ever met a person you think lied about working...,You ever get the feeling someone online or in-...,NerdyMcDataNerd,1.745249e+09,71,73,0.84
3,1k3nxj7,"Pandas, why the hype?",I'm an R user and I'm at the point where I'm n...,gonna_get_tossed,1.745160e+09,335,190,0.87
4,1k3e4nb,Unit tests,Serious question: Can anyone provide a real ex...,genobobeno_va,1.745122e+09,31,22,0.82


In [21]:
# Fetch comments for all posts in posts_df_reddit
all_comments = []

for post_id in posts_df_reddit['post_id']:
    try:
        comments_df = get_post_comments(reddit, post_id, limit=100)
        all_comments.append(comments_df)
        print(f"✅ Retrieved {len(comments_df)} comments for post: {post_id}")
    except Exception as e:
        print(f"❌ Failed to fetch comments for {post_id}: {e}")

# Combine all comments into a single DataFrame
all_comments_df = pd.concat(all_comments, ignore_index=True)

# Save to CSV
all_comments_df.to_csv("../reddit_data/reddit_post_comments.csv", index=False)
print("💾 All comments saved to reddit_data/reddit_post_comments.csv")


✅ Retrieved 1 comments for post: 1k44mgg
✅ Retrieved 24 comments for post: 1i5inrb
✅ Retrieved 39 comments for post: 1k4geso
✅ Retrieved 93 comments for post: 1k3nxj7
✅ Retrieved 4 comments for post: 1k3e4nb
✅ Retrieved 23 comments for post: 1k32lrl
✅ Retrieved 1 comments for post: 1k3jt7b
✅ Retrieved 13 comments for post: 1k2y84g
✅ Retrieved 2 comments for post: 1k33k6t
✅ Retrieved 7 comments for post: 1k35lig
✅ Retrieved 12 comments for post: 1k2igce
✅ Retrieved 1 comments for post: 1k2u4nd
✅ Retrieved 32 comments for post: 1k26kp3
✅ Retrieved 4 comments for post: 1k2a8t6
✅ Retrieved 34 comments for post: 1k26920
✅ Retrieved 9 comments for post: 1k2ax74
✅ Retrieved 19 comments for post: 1k22cd4
✅ Retrieved 13 comments for post: 1k1wu9o
✅ Retrieved 11 comments for post: 1k1mjok
✅ Retrieved 4 comments for post: 1k1x464
✅ Retrieved 1 comments for post: 1k1vo23
✅ Retrieved 4 comments for post: 1k1lh3r
✅ Retrieved 1 comments for post: 1k1ohsp
✅ Retrieved 5 comments for post: 1k20azb
✅ Ret

In [13]:
all_comments_df.head()

,comment_id,post_id,author,text,score,created_utc
0,m8l6gbd,1i5inrb,dogdiarrhea,Nonstandard transition question: how many peop...,5,1.737573e+09
1,m8wl92v,1i5inrb,Left-Animal1559,I am a senior talent partner with Swish Analyt...,6,1.737725e+09
2,m8y2o1w,1i5inrb,doorstoinfinity,Hi everyone!\n\nI'm transitioning from Data An...,4,1.737741e+09
3,m84ed38,1i5inrb,Silent_Group6621,Can someone suggest me some DS/Analytics/ML pr...,3,1.737354e+09
4,m85eg48,1i5inrb,j-unnlock,Here's a list of job boards I'm working on ser...,2,1.737376e+09
